# 처음부터 ReAct 에이전트를 만드는 방법 (함수형 API)

!!! info "필수 조건"
    이 가이드는 다음에 대한 익숙함을 가정합니다:
    
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models)
    - [메시지](https://python.langchain.com/docs/concepts/messages)
    - [도구 호출](https://python.langchain.com/docs/concepts/tool_calling/)
    - [엔트리포인트](../../concepts/functional_api/#entrypoint) 및 [태스크](../../concepts/functional_api/#task)

이 가이드는 LangGraph [함수형 API](../../concepts/functional_api)를 사용하여 ReAct 에이전트를 구현하는 방법을 보여줍니다.

ReAct 에이전트는 다음과 같이 작동하는 [도구 호출 에이전트](../../concepts/agentic_concepts/#tool-calling-agent)입니다:

1. 쿼리가 채팅 모델에 발행됩니다;
2. 모델이 [도구 호출](../../concepts/agentic_concepts/#tool-calling)을 생성하지 않으면 모델 응답을 반환합니다.
3. 모델이 도구 호출을 생성하면 사용 가능한 도구로 도구 호출을 실행하고, [도구 메시지](https://python.langchain.com/docs/concepts/messages/)로 메시지 목록에 추가하고, 프로세스를 반복합니다.

이것은 메모리, 사람-인-더-루프 기능 및 기타 기능으로 확장할 수 있는 간단하고 다재다능한 설정입니다. 예제는 전용 [how-to 가이드](../../how-tos/#prebuilt-react-agent)를 참조하세요.

## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정하겠습니다:

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

<div class="admonition tip">
     <p class="admonition-title">더 나은 디버깅을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
     <p style="padding-top: 5px;">
         LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">문서</a>를 참조하세요. 
     </p>
 </div>

## ReAct 에이전트 생성

이제 필요한 패키지를 설치하고 환경 변수를 설정했으므로 에이전트를 만들 수 있습니다.

### 모델 및 도구 정의

먼저 예제에 사용할 도구와 모델을 정의하겠습니다. 여기서는 위치의 날씨 설명을 가져오는 단일 플레이스홀더 도구를 사용합니다.

이 예제에서는 [OpenAI](https://python.langchain.com/docs/integrations/providers/openai/) 채팅 모델을 사용하지만 [도구 호출을 지원하는](https://python.langchain.com/docs/integrations/chat/) 모든 모델이 충분합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """특정 위치의 날씨를 가져오기 위해 호출합니다."""
    # 이것은 실제 구현을 위한 플레이스홀더입니다
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]

### 태스크 정의

다음으로 실행할 [태스크](../../concepts/functional_api/#task)를 정의합니다. 여기에는 두 가지 다른 태스크가 있습니다:

1. **모델 호출**: 메시지 목록으로 채팅 모델을 쿼리하려고 합니다.
2. **도구 호출**: 모델이 도구 호출을 생성하면 실행하려고 합니다.

In [ ]:
from langchain_core.messages import ToolMessage
from langgraph.func import entrypoint, task

tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """메시지 시퀀스로 모델을 호출합니다."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])

### 엔트리포인트 정의

우리의 [엔트리포인트](../../concepts/functional_api/#entrypoint)는 이 두 태스크의 오케스트레이션을 처리할 것입니다. 위에서 설명한 대로 `call_model` 태스크가 도구 호출을 생성하면 `call_tool` 태스크가 각각에 대한 응답을 생성합니다. 모든 메시지를 단일 메시지 목록에 추가합니다.

!!! tip
    태스크가 future와 같은 객체를 반환하기 때문에 아래 구현은 도구를 병렬로 실행합니다.

In [ ]:
from langgraph.graph.message import add_messages


@entrypoint()
def agent(messages):
    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # 도구 실행
        tool_result_futures = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]

        # 메시지 목록에 추가
        messages = add_messages(messages, [llm_response, *tool_results])

        # 모델 다시 호출
        llm_response = call_model(messages).result()

    return llm_response

## 사용

에이전트를 사용하려면 메시지 목록으로 호출합니다. 구현에 따라 LangChain [메시지](https://python.langchain.com/docs/concepts/messages/) 객체 또는 OpenAI 스타일 딕셔너리가 될 수 있습니다:

In [ ]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message]):
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # 태스크 업데이트만 출력
        print(f"\n{task_name}:")
        message.pretty_print()

완벽합니다! 그래프가 `get_weather` 도구를 올바르게 호출하고 도구에서 정보를 받은 후 사용자에게 응답합니다. LangSmith 추적을 [여기](https://smith.langchain.com/public/d5a0d5ea-bdaa-4032-911e-7db177c8141b/r)에서 확인하세요.

## 스레드 수준 지속성 추가

[스레드 수준 지속성](../../concepts/persistence#threads)을 추가하면 에이전트와의 대화 경험을 지원할 수 있습니다: 후속 호출은 이전 메시지 목록에 추가되어 전체 대화 컨텍스트를 유지합니다.

에이전트에 스레드 수준 지속성을 추가하려면:

1. [체크포인터](../../concepts/persistence#checkpointer-libraries) 선택: 여기서는 간단한 인메모리 체크포인터인 [InMemorySaver](../../reference/checkpoints/#langgraph.checkpoint.memory.InMemorySaver)를 사용합니다.
2. 이전 메시지 상태를 두 번째 인수로 받도록 엔트리포인트를 업데이트합니다. 여기서는 메시지 업데이트를 이전 메시지 시퀀스에 단순히 추가합니다.
3. 워크플로우에서 반환될 값과 `entrypoint.final`을 사용하여 체크포인터에 의해 `previous`로 저장될 값을 선택합니다 (선택 사항)

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# highlight-next-line
checkpointer = InMemorySaver()


# highlight-next-line
@entrypoint(checkpointer=checkpointer)
# highlight-next-line
def agent(messages, previous):
    # highlight-next-line
    if previous is not None:
        # highlight-next-line
        messages = add_messages(previous, messages)

    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # 도구 실행
        tool_result_futures = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]

        # 메시지 목록에 추가
        messages = add_messages(messages, [llm_response, *tool_results])

        # 모델 다시 호출
        llm_response = call_model(messages).result()

    # 최종 응답 생성
    messages = add_messages(messages, llm_response)
    # highlight-next-line
    return entrypoint.final(value=llm_response, save=messages)

이제 애플리케이션을 실행할 때 config를 전달해야 합니다. config는 대화 스레드의 식별자를 지정합니다.

!!! tip

    스레드 수준 지속성에 대해 자세히 알아보려면 [개념 페이지](../../concepts/persistence/)와 [how-to 가이드](../../how-tos/#persistence)를 참조하세요.

In [6]:
config = {"configurable": {"thread_id": "1"}}

이전과 같은 방식으로 스레드를 시작하지만 이번에는 config를 전달합니다:

In [ ]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

# highlight-next-line
for step in agent.stream([user_message], config):
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # 태스크 업데이트만 출력
        print(f"\n{task_name}:")
        message.pretty_print()

후속 대화를 물어보면 모델은 이전 컨텍스트를 사용하여 날씨에 대해 묻고 있음을 추론합니다:

In [ ]:
user_message = {"role": "user", "content": "How does it compare to Boston, MA?"}
print(user_message)

for step in agent.stream([user_message], config):
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # 태스크 업데이트만 출력
        print(f"\n{task_name}:")
        message.pretty_print()

[LangSmith 추적](https://smith.langchain.com/public/20a1116b-bb3b-44c1-8765-7a28663439d9/r)에서 각 모델 호출에서 전체 대화 컨텍스트가 유지되는 것을 확인할 수 있습니다.